# Coordinate Systems

## Overview 
**This tutorial covers:**

- The relationsship of different coordinate systems towards each other and how they are handled by WelDX

**Requirements:**

Opening and navigating through WelDX files
 TODO: link tutorial, once it is merged

## Dependency graph

All spatial data like specimen geometries or the trajectory of a welding torch is defined in a coordinate system that serves as anchor point.
But this isn't necessarily always the same coordinate system.
In fact, it is often more convinient to use different coordinate systems for individual data sets.
This raises the problem that we need to know how the different systems are oriented towards each other.
WelDX ensures this by a special tree like structure called the `CoordinateSystemManager` (we will use the abbrevation CSM from here on) that need to be part of almost every WelDX file.
Let's find it in our example file and extract it to see what we can do with it:

In [ ]:
from weldx import WeldxFile
wxfile = WeldxFile("single_pass_weld.wx")

In [ ]:
wxfile.info()

Examining the `info` output, we find that the object we are looking for is stored under the key `"coordinate_systems"`.

In [ ]:
csm = wxfile["coordinate_systems"]

As previously mentioned, the CSM is based on a tree structure.
Every element added to it needs to define the transformation between itself and its parent node.
This ensures in combination with the tree structure that there is always a transformation path between all contained coordinate systems.
We can get a nice overview of all coordinate systems and their relationship towards each other by calling the `plot_graph` method:

In [ ]:
csm.plot_graph()

> HINT: In a jupyter session it is sufficient to just type the variable name at the end of a cell (here `csm`). 
  This will also plot the graph

The plot shows us multiple things.
The most important information are the coordinate system names.
Arrows indicate that there is a transformation defined between two coordinate systems.
The direction of the arrows has no practical relevance for using the CSM.
It just specifies the transformation direction that was originally provided by the creator of the file.
Becuase we can always calculate the inverse transformation, it can be considered as pure information.
More important is the color of the arrows.
Black arrows indicate that the transformation remains constant during the experiment. 
For example, the coordinate system of a thermal sensor (`T1`) that is attached to a fixed position on the specimen will not change its relative position towards the specimen coordinate system (`workpiece`).
Therefore, the transformation remains constant.
Yellow arrows represent time dependent transformations.
A good for example for this is the torch in a welding application (`TCP`).
During an experiment, it moves in relation to the workpiece.

## 3d plot and time interpolation

The graph plot is nice to get a quick overview of the defined coordinate systems and how they depend on each other, but it does't provide us any information on how they are arranged in 3d space.
For this purpose, the CSM also has a `plot` method.
But before we use this method we have to take a small detour and talk about how time dependencies are treated by the CSM.

The number of data points for the torch trajectory (TCP) is relatively high because we are dealing with real measurement data here that was recorded with a high frequency.
When using the default rendering backend (matplotlib) each time dependent data point is rendered individually.
With huge amount of data, this takes a rather long time to compute and will possibly clutter the image with data points so that is hard to recognize anything.
The solution to both problems is to resample the time dependent data of the CSM.

We do this in three steps.
First we get the `time_union` of the CSM:

In [ ]:
time_union = csm.time_union()

The `time_union` method collects every time value of all the time dependent coordinate systems of the CSM and merges them into a `Time` object.
`Time` is a simple class that handles all time related operations.
One of it's methods is called `resample`.
With this method we can create a new `Time` object with a specified number of data points that still has the same time boundaries as the original one.
So let's say we want to plot our time dependen data with 10 time steps.
We create a corresponding `Time` object with:

In [ ]:
time_resampled = time_union.resample(10)

We pass the the resampled time object to the `interp_time` method of the CSM.
This will create a new CSM instace where all time dependent data is interpolated to the passed time values:

In [ ]:
csm_interp = csm.interp_time(time_resampled)

Now we can call `plot` of the interpolated CSM without having to worry, that the calculation takes several minutes to create the plot.
Note that we use `data_sets=[]` as additional argument.
This supresses the rendering of the attached specimen geometry data that would clutter the plot.

In [ ]:
# uncomment next line to enable interactive plots
# %matplotlib widget

In [ ]:
csm_interp.plot(data_sets=[])

The plot shows us the locations of all coordinate systems in 3d space.
Time dependent coordinate systems are represented by multiple control points that are connected by an equally colored line.
Each control point marks the position at one of the time steps we used to interpolate the original CSM.

Depending on the number of coordinate systems, it might be beneficial to render just the ones we are interested in.
For this purpose, we can use the `coordinate_systems` parameter.
It expects a `list` with the names of the coordinate systems we like to render: 

In [ ]:
csm_interp.plot(
    coordinate_systems=["user_frame", "TCP", "flange"],
    data_sets=[]
)

By checking the coordinate $x=0$, $y=0$, $z=0$ in the plot, you can derive that all data is plotted in the "user_frame" coordinate system.
This is because it is the root coordinate system of the CSM as can be seen in the graph we plotted earlier.
If you want to change the reference coordinate system, you can do this with the parameter `reference_system`.
This is especially interesting if time dependencies are involved.
Let's see how everything looks how everything looks from the torchs' perspective:

In [ ]:
csm_interp.plot(
    coordinate_systems=["user_frame", "TCP", "flange"],
    data_sets=[],
    reference_system="TCP"
)

Now the "user_frame" has suddenly become time dependent and the flange remains static.
This does make sense considering that the torch is mounted on the flange and both move in relation to the "user_frame" coordinate system.
The `plot` method has a lot more parameters to tweak the plot to your liking.
However, demonstrating them all would exceed the scope of this tutorial.
Checkout the [corresponding API documentation page](https://weldx.readthedocs.io/en/latest/_autosummary/weldx.CoordinateSystemManager.plot.html#weldx.CoordinateSystemManager.plot) to learn more.